# Módulo de repaso Visión Artificial 2020-02

Leamos las librerías útiles

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math

#En este notebook se define una función que facilita la lectura de imágenes
#Leer la imagen en RGB o escala de grises con una sola línea de código
def img_read(filename, mode = 'color'):
    if(mode == 'color'):   
        return cv2.cvtColor(cv2.imread(filename),cv2.COLOR_BGR2RGB)
    elif(mode=='grayscale' or mode=='greyscale' or mode == 'gray' or mode == 'grey'):
        return cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
        return None

Las primeras pinceladas del trabajo en Visión Artificial están basadas en el análisis de color de un problema dado, no siendo éste el único método de preprocesamiento y segmentación de elementos característicos, es muy intuitivo y relativamente sencillo de implementar.

A continuación, observemos un poco del alcance de éste método. Supongamos que tenemos a nuestro alcance unas pelotas/círculos de distintos colores. ¿Cómo se observarían éstas en diferentes espacios de color?

In [ ]:
#Leer y plotear la imagen de trabajo
img = cv2.imread("res/colores.jpg", cv2.IMREAD_COLOR)
img_rgb  = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)

In [ ]:
img_R = img_rgb[:,:,0]
img_G = img_rgb[:,:,1]
img_B = img_rgb[:,:,2]
img_lab  = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
img_L = img_lab[:,:,0]
img_A =  img_lab[:,:,1]
img_BB = img_lab[:,:,2]
fig, arreglo_plots = plt.subplots(6,2, figsize=(15, 25))
colors = ('r','g','b')

#Canal LAB
arreglo_plots[0,0].set_title('Canal R')
arreglo_plots[0,0].imshow(img_R, cmap="gray")
arreglo_plots[0,1].set_title('Histograma Canal R')
img_array_i = img_rgb[:,:,0].ravel()
arreglo_plots[0,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[0])

arreglo_plots[1,0].set_title('Canal G')
arreglo_plots[1,0].imshow(img_G, cmap="gray")
arreglo_plots[1,1].set_title('Histograma Canal G')
img_array_i = img_rgb[:,:,1].ravel()
arreglo_plots[1,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[1])
    
arreglo_plots[2,0].set_title('Canal B')
arreglo_plots[2,0].imshow(img_B, cmap="gray")
arreglo_plots[2,1].set_title('Histograma Canal B')
img_array_i = img_rgb[:,:,2].ravel()
arreglo_plots[2,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[2])

#Canal XYZ
arreglo_plots[3,0].set_title('Canal L')
arreglo_plots[3,0].imshow(img_L, cmap="gray")
arreglo_plots[3,1].set_title('Histograma Canal L')
img_array_i = img_lab[:,:,0].ravel()
arreglo_plots[3,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[0])

arreglo_plots[4,0].set_title('Canal A')
arreglo_plots[4,0].imshow(img_A, cmap="gray")
arreglo_plots[4,1].set_title('Histograma Canal A')
img_array_i = img_lab[:,:,1].ravel()
arreglo_plots[4,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[1])
    
arreglo_plots[5,0].set_title('Canal B')
arreglo_plots[5,0].imshow(img_BB, cmap="gray")
arreglo_plots[5,1].set_title('Histograma Canal B*')
img_array_i = img_lab[:,:,2].ravel()
arreglo_plots[5,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[2]);

## Recordemos.

### ¿Si quisieramos separar las pelotas amarillas de la imagen, qué canal podríamos utilizar?

### Así mismo, ¿Hay algún contenido de rojo en los circulos cyan?

### ¿Sería fácil aplicar una segmentación de tipo threshold sobre el canal B para separar  el circulo azul de los demás? ¿Por qué?

### Como ejercicio rápido, probar el canal HSV y comentar el resultado.

Por otro lado, hemos recorrido las transformaciones lineales y no-lineales sobre las imágenes.

## Suma ponderada.

In [ ]:
#Leer las imágenes, hacerlas del mismo tamaño para poderlas sumar
img1 = cv2.cvtColor(cv2.imread('res/gato.jpg'), cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread('res/boca.png'), cv2.COLOR_BGR2RGB)
img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

#Indicamos el parámetro a que indica la importancia de cada imagen
a = 0.5

#Mediante la T.Lineal, si a=1, tenemos la imagen 1 únicamente. Si a=0, obtenemos solo img2.
# Si 0<a<1 tendremos combinaciones ponderadas de las imágenes entrada
res = cv2.add(img1*a,img2*(1-a)).astype(np.uint8)

#Crear una figura 1x3 para mostrar las imágenes de entrada y la combinación ponderada
fig = plt.figure(figsize=(15,24))

plt.subplot(1,3,1)
plt.imshow(img1,cmap='gray')
plt.title("Imagen 1")

plt.subplot(1,3,2)
plt.imshow(img2,cmap='gray')
plt.title("Imagen 2")

plt.subplot(1,3,3)
plt.imshow(res,cmap='gray')
plt.title("Suma");

# Resaltar bordes

In [ ]:
#Leer la imagen de entrada
img1 =cv2.cvtColor(cv2.imread('res/gato.jpg'), cv2.COLOR_BGR2RGB)

#Utilizamos la función cv2.Canny, que corresponde a un algoritmo de detección de bordes
edges = cv2.Canny(img1,100,200)
#Resaltar los bordes de blanco si sus pixeles son menores que un valor umbral
edges[edges<128] = 0
#Convertir la imagen edges a RGB
edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
edges[:,:,0]=0
edges[:,:,1]=0               #para obtener el color magenta (255,0,255)

#Crear una figura 1x3 para mostrar la imagen de entrada, la imagen obtenida con edges y la superposición
fig = plt.figure(figsize=(15,24))

plt.subplot(1,3,1)
plt.imshow(img1,cmap='gray')
plt.title("Imagen 1")

plt.subplot(1,3,2)
plt.imshow(edges,cmap='gray')
plt.title("Imagen 2")

plt.subplot(1,3,3)
plt.imshow(cv2.add(img1,edges),cmap='gray')
plt.title("Imagen 3");

## Analicemos.
## ¿Cómo se verían los bordes de la imagen gato2.jpg?

# Resta.

In [ ]:
#Lectura de las dos imágenes
img1 = cv2.cvtColor(cv2.imread('res/correr.png'), cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread('res/correr2.png'), cv2.COLOR_BGR2RGB)

#Conversión a tipo de dato uint16
img1 = img1.astype(np.uint16)
img2 = img2.astype(np.uint16)

#Suma 125 unidades a img1, hace la resta de imágenes y los extraemos de nuevo
img1 += 125
res = img1 - img2
img1 -= 125

#Regresa al tipo de dato uint8 entre 0 y 255
img1 = img1.astype(np.uint8)
img2 = img2.astype(np.uint8)
res=res.astype(np.uint8)

#Genera una figura 1x3 para mostrar la imagen, una estimacion de fondo y la resta
fig = plt.figure(figsize=(15,24))

plt.subplot(1,3,1)
plt.imshow(img1,cmap='gray')
plt.title("Imagen")

plt.subplot(1,3,2)
plt.imshow(img2,cmap='gray')
plt.title("Imagen tras el movimiento")

plt.subplot(1,3,3)
plt.imshow(res,cmap='gray')
plt.title("Resta");

## Analicemos.

## Comparemos la imagen "res/correr.png" y "res/correr3.png".

# Multiplicación.

In [ ]:
#Leer la imagen
img = cv2.cvtColor(cv2.imread('res/gato2.jpg'), cv2.COLOR_BGR2RGB)

#Leer la máscara y normalizarla, cambiar su tipo de dato y ajustar su tamaño
mascara = cv2.imread('res/mascara.png')/255
mascara = cv2.resize(mascara, (img.shape[1], img.shape[0])).astype(np.uint8)

#Operación multiplicar
out = cv2.multiply(img, mascara)

#Figura 1x3 para mostrar imagen de entrada, máscara y la multiplicación
fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(15, 7))
ax1.set_title('Imagen de entrada')
ax2.set_title('Máscara')
ax3.set_title('Multiplicación')
ax1.imshow(img, cmap='gray')
ax2.imshow(mascara, cmap='gray', vmin = 0, vmax = 1)
ax3.imshow(out, cmap='gray')

In [ ]:
print(np.max(mascara))

## Analicemos.
## ¿Qué ocurre si aplicamos la misma máscara a la imagen gato.jpg?

# División.

A continuación comparemos el resultado entre las imágenes "res/correr.png" y "res/correr2.png", obtenido por resta y por división.

In [ ]:
#Lectura de imágenes de entrada, se ponen del mismo tamaño para poder operarlas
img1 = cv2.cvtColor(cv2.imread('res/correr.png'), cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread('res/correr2.png'), cv2.COLOR_BGR2RGB)
img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))


#RESTA
img1 = img1.astype(np.uint16)
img2 = img2.astype(np.uint16)
img1 += 125
res = img1 - img2
res = res.astype(np.uint8)
img1 -= 125
img1 = img1.astype(np.uint8)
img2 = img2.astype(np.uint8)

#DIVISION

#Es importante convertir los datos a flotante si se dispone a dividir
img1_div = img1.astype(np.float32)
img2_div = img2.astype(np.float32)
div = cv2.divide(img1_div,img2_div)
div = div * 100
div = div.astype(np.uint8)

#Crear figura 2x2 para mostrar las imágenes de entrada con sus operaciones respectivas
fig = plt.figure(figsize=(15,10))

plt.subplot(2,2,1)
plt.imshow(img1,cmap='gray')
plt.title("Imagen 1")

plt.subplot(2,2,2)
plt.imshow(img2,cmap='gray')
plt.title("Imagen 2")

plt.subplot(2,2,3)
plt.imshow(res,cmap='gray')
plt.title("Resta")

plt.subplot(2,2,4)
plt.imshow(div,cmap='gray')
plt.title("División")

## Analicemos
## Repitamos el mismo ejercicio de la resta, aplicando división.

Por otro lado, existen transformaciones geométricas sobre las imágenes. Recordemos cuáles son.

# Traslación.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

#Lectura de la imagen en escala de grises
img =cv2.imread('res/gato.jpg',cv2.IMREAD_GRAYSCALE)
#Se obtienen los vectores filas, columnas
filas,columnas = img.shape
#Se asignan valores a tx y ty, se construye la matriz M de translación
tx = 15
ty = 15
matriz_afin = np.float32([[1,0,tx],
                          [0,1,ty]])

#Transformación de la imagen con base en la matriz M, usando warpAffine
res = cv2.warpAffine(img,matriz_afin,(columnas,filas))# , flags=cv2.INTER_CUBIC)

#Figura 1x2 para mostrar la entrada y la salida
fig = plt.figure(figsize=(10,12))

plt.subplot(1,2,1)
plt.imshow(img,cmap='gray')
plt.title("Imagen Original")

plt.subplot(1,2,2)
plt.imshow(res,cmap='gray')
plt.title("Imagen Salida")


## Analicemos.
## Corrija la traslación, para poder observar la cara del gato en la imagen trasladada.

# Rotación.

Es posible rotar la imagen cierta cantidad de grados utilizando las matrices correctas.

In [ ]:
#Lectura de la imagen de entrada
img =cv2.imread('res/gato2.jpg',cv2.IMREAD_GRAYSCALE)
#Obtener los datos de filas y columnas de la imagen
filas,columnas = img.shape

#Asignar un ángulo de rotación
theta = 30

#Selección del eje de rotación
rotation_center = ((columnas/2,filas/2))
#rotation_center = ((0,0))

#Con base en el ángulo, utilizar getRotationMatrix2D de cv2
matriz_afin_rot = cv2.getRotationMatrix2D(rotation_center,theta,1)
#Mostrar la matriz calculada
print(matriz_afin_rot)

#Aplicar la matriz obtenida sobre la imagen leída
rot = cv2.warpAffine(img,matriz_afin_rot,(columnas,filas))

#Figura 1x2 para mostrar imagen original y resultados
fig = plt.figure(figsize=(10,12))

plt.subplot(1,2,1)
plt.imshow(img,cmap='gray')
plt.title("Imagen Original")

plt.subplot(1,2,2)
plt.imshow(rot,cmap='gray')
plt.title("Imagen Salida");

## Analicemos.
## Rote la imagen original con distintos ángulos (positivos o negativos), comente.

# Inclinación y cambios de escala.

Probemos qué sucede si hacemos un cambio de escala en la imagen, aproximando valores para no perder tanta información.

Adicionalmente, observemos qué sucede si queremos inclinar la imagen de entrada, una transformación geométrica utilizando otra matriz adecuada.

In [ ]:
#Lectura de imagen de entrada y obtención de filas y columnas
img =cv2.imread('res/gato.jpg',cv2.IMREAD_GRAYSCALE)
filas,columnas = img.shape

#Con base en la teoría, crear la matriz M de inclinación
#reemplazando los valores b y d
inc_x = 0.0
inc_y = 0.5
matriz_afin = np.float32([[1,inc_x,0],
                          [inc_y,1,0]])

#Aplicar la matriz generada a la imagen leída
res = cv2.warpAffine(img,matriz_afin,(columnas,filas))

#Generar la figura 1x2 y mostrar entrada vs resultados
fig = plt.figure(figsize=(10,12))

plt.subplot(1,2,1)
plt.imshow(img,cmap='gray')
plt.title("Imagen Original")

plt.subplot(1,2,2)
plt.imshow(res,cmap='gray')
plt.title("Imagen Salida")

In [ ]:
#Leer la imagen de entrada a color y convertirla RGB
img=cv2.imread('res/gato.jpg',cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#Acercar la imagen de entrada con base en una aproximación de vecinos cercanos
#o interpolación bilinear/bicúbica
nearest = cv2.resize(img,(img.shape[1]*4, img.shape[0]*4), interpolation = cv2.INTER_NEAREST)
bilinear = cv2.resize(img,None,fx=4, fy=4, interpolation = cv2.INTER_LINEAR)
bicubic = cv2.resize(img,None,fx=4, fy=4, interpolation = cv2.INTER_CUBIC)

#Generar figura 2x2 para mostrar y contrastar las aproximaciones con la imagen original
fig = plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
plt.imshow(img,cmap='gray')
plt.title("Imagen Original")

plt.subplot(2,2,2)
plt.imshow(nearest,cmap='gray')
plt.title("Interpolación vecinos más cercanos")

plt.subplot(2,2,3)
plt.imshow(bilinear,cmap='gray')
plt.title("Interpolación bilineal")

plt.subplot(2,2,4)
plt.imshow(bilinear,cmap='gray')
plt.title("Interpolación bicúbica")

#Guardar las imágenes en la carpeta del notebook
nearest = cv2.cvtColor(nearest, cv2.COLOR_BGR2RGB)
bilinear = cv2.cvtColor(bilinear, cv2.COLOR_BGR2RGB)
bicubic = cv2.cvtColor(bicubic, cv2.COLOR_BGR2RGB)


cv2.imwrite("nearest.jpg", nearest);
cv2.imwrite("bilinear.jpg", bilinear);
cv2.imwrite("bicubic.jpg", bicubic);

## Analicemos.
## Pruebe inclinando la imagen gato2.jpg con parámetros 0.4 en X y -0.5 en Y.
## Realice la interpolación utilizando la imagen gato2.jpg

## ¿Hay alguna diferencia notoria con estas aproximaciones?

# Rápidamente, recordemos suma, resta, multiplicación y división sobre el histograma. Y algún caso especial de éstas transformaciones.

# Suma

In [ ]:
#Se realiza la lectura de la imagen
img = cv2.cvtColor(cv2.imread('res/gato.jpg'), cv2.COLOR_BGR2RGB)
#img_add es una matriz de ceros del mismo tamaño de la imagen original, corresponde a una copia para editar.
#el tipo de dato debe ser uint8 para estar entre 0 y 255
img_add = np.zeros(img.shape, np.uint8)

#A cada uno de los canales de img se le suman q,w,e unidades. 
q = 50 ; w = 50; e = 50
#Se sustituyen los valores de cada canal de la copia por los valores modificados de la imagen original
img_add[:,:,0] = cv2.add(img[:,:,0], q)
img_add[:,:,1] = cv2.add(img[:,:,1], w)
img_add[:,:,2] = cv2.add(img[:,:,2], e)

#Se realiza el arreglo de 2x2 para mostrar imagen e imagen modificada vs histogramas
fig, arreglo_plots = plt.subplots(2,2, figsize=(15, 15))
colors = ('r','g','b')

#Se le dan títulos a los cuadros, se imprimen las imágenes y sus histogramas
arreglo_plots[0,0].set_title('Imagen original')
arreglo_plots[0,0].imshow(img, cmap="gray")
arreglo_plots[0,1].set_title('Histograma original')
for i,col in enumerate(colors):
    img_array_i = img[:,:,i].ravel()
    arreglo_plots[0,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),normed=True, color=colors[i])

arreglo_plots[1,0].set_title('Imagen modificada')
arreglo_plots[1,0].imshow(img_add, cmap="gray")
arreglo_plots[1,1].set_title('Histograma modificado')
for i,col in enumerate(colors):
    img_array_i = img_add[:,:,i].ravel()
    arreglo_plots[1,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),normed=True, color=colors[i])

# Resta

In [ ]:
#Se lee la imagen 
img = cv2.cvtColor(cv2.imread('res/gato2.jpg'), cv2.COLOR_BGR2RGB)
#Se crea la imagen copia para editarla
img_subtract = np.zeros(img.shape, np.uint8)
#Se restan q,w,e unidades a los canales de img.
#Se sustituyen los valores de cada canal de la copia por los valores modificados de la imagen original
q = 50; w = 50; e = 50
img_subtract[:,:,0] = cv2.subtract(img[:,:,0], q)
img_subtract[:,:,1] = cv2.subtract(img[:,:,1], w)
img_subtract[:,:,2] = cv2.subtract(img[:,:,2], e)

#Se realiza el arreglo de 2x2 para mostrar imagen e imagen modificada vs histogramas
fig, arreglo_plots = plt.subplots(2,2, figsize=(15, 15))
colors = ('r','g','b')

#Se le dan títulos a los cuadros, se imprimen las imágenes y sus histogramas
arreglo_plots[0,0].set_title('Imagen original')
arreglo_plots[0,0].imshow(img, cmap="gray")
arreglo_plots[0,1].set_title('Histograma original')
for i,col in enumerate(colors):
    img_array_i = img[:,:,i].ravel()
    arreglo_plots[0,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[i])

arreglo_plots[1,0].set_title('Imagen modificada')
arreglo_plots[1,0].imshow(img_subtract, cmap="gray")
arreglo_plots[1,1].set_title('Histograma modificado')
for i,col in enumerate(colors):
    img_array_i = img_subtract[:,:,i].ravel()
    arreglo_plots[1,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[i])

## Analicemos.
## ¿Qué modifican los valores de las variables q,w,e? ¿Qué ocurre si al modificarlos, en suma y resta, q>>w & q >>e

# Multiplicación/División

In [ ]:
#Se lee la imagen
img = cv2.cvtColor(cv2.imread('res/gato.jpg'), cv2.COLOR_BGR2RGB)
#Se crea la imagen copia
img_mult = np.zeros(img.shape, np.uint8)

#Se multiplican los valores de cada canal de img por q,w,e unidades
#Se sustituyen los valores de cada canal de la copia por los valores modificados de la imagen original
q = 2; w = 2; e = 2
img_mult[:,:,0] = cv2.multiply(img[:,:,0], q)
img_mult[:,:,1] = cv2.multiply(img[:,:,1], w)
img_mult[:,:,2] = cv2.multiply(img[:,:,2], e)

#Se realiza el arreglo de 2x2 para mostrar imagen e imagen modificada vs histogramas
fig, arreglo_plots = plt.subplots(2,2, figsize=(15, 15))
colors = ('r','g','b')

#Se le dan títulos a los cuadros, se imprimen las imágenes y sus histogramas
arreglo_plots[0,0].set_title('Imagen original')
arreglo_plots[0,0].imshow(img, cmap="gray")
arreglo_plots[0,1].set_title('Histograma original')
for i,col in enumerate(colors):
    img_array_i = img[:,:,i].ravel()
    arreglo_plots[0,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[i])

arreglo_plots[1,0].set_title('Imagen modificada')
arreglo_plots[1,0].imshow(img_mult, cmap="gray")
arreglo_plots[1,1].set_title('Histograma modificado')
for i,col in enumerate(colors):
    img_array_i = img_mult[:,:,i].ravel()
    arreglo_plots[1,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[i])

## Analicemos.
## Modifique el código anterior y obtenga la operación división. De nuevo, varíe de forma antisimétrica los valores de q,w,e.

# La transformación Gamma.

La corrección Gamma tienes su raíces en la forma en que los humanos percibimos la luz y el color. Esta percepción es de tipo no lineal y en condiciones de iluminación normal sigue un comportamiento de potencia.

Gamma puede adquirir valores 0 < gamma < 1 y aparentar una raíz. Así mismo, puede ser gamma > 1 y actuar como una potencia.

In [ ]:
#Definir la función para aplicar la transformación sobre la imagen RGB
def apply_f_on_rgb(img, f, args):
    
    #Crear una matriz de ceros del tamaño de la imagen de entrada
    res = np.zeros(img.shape, np.uint8)
    #Aplicar la transformación f sobre cada canal del espacio de color RGB
    res[:,:,0] = f(img[:,:,0], *args)
    res[:,:,1] = f(img[:,:,1], *args)
    res[:,:,2] = f(img[:,:,2], *args)
    
    return res

#Definir la función de transformación de la imagen (corrección gamma)
def gamma_correction(img, a, gamma):
    
    #Crear copia de la imagen tipo flotante dada la normalización
    img_copy = img.copy().astype(np.float32)/255.0
    #La función corrección gamma es de la forma ax^gamma, donde x es la imagen de entrada
    res_gamma = cv2.pow(img_copy,gamma)
    res = cv2.multiply(res_gamma, a)
    
    #Asegurar que la los datos queden entre 0 y 255 y sean uint8
    res[res<0] = 0
    res = res*255.0
    res[res>255] = 255
    
    res = res.astype(np.uint8)
    
    return res

#Leer la imagen de entrada
img = cv2.cvtColor(cv2.imread('res/gato.jpg'), cv2.COLOR_BGR2RGB)

#Dar valor a los parámetros a,gamma (args)
a = 1
gamma = 0.2

#Aplicar la transformación corrección gamma sobre la imagen de entrada
res_brillante = apply_f_on_rgb(img, gamma_correction, [a, gamma])

#Variar los parámetros a y gamma y volver a aplicar la corrección
a = 1
gamma = 2

res_oscuro = apply_f_on_rgb(img, gamma_correction, [a, gamma])

#Generar un subplot 2x2 para contrastar imágen original con transformada y sus histogramas
fig, arreglo_plots = plt.subplots(3,2, figsize=(15, 25))
colors = ('r','g','b')

arreglo_plots[0,0].set_title('Imagen original')
arreglo_plots[0,0].imshow(img, cmap="gray")
arreglo_plots[0,1].set_title('Histograma original')
for i,col in enumerate(colors):
    img_array_i = img[:,:,i].ravel()
    arreglo_plots[0,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[i])

arreglo_plots[1,0].set_title('Imagen con incremento de brillo')
arreglo_plots[1,0].imshow(res_brillante, cmap="gray")
arreglo_plots[1,1].set_title('Histograma modificado')
for i,col in enumerate(colors):
    img_array_i = res_brillante[:,:,i].ravel()
    arreglo_plots[1,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[i])
    
arreglo_plots[2,0].set_title('Imagen con reducción de brillo')
arreglo_plots[2,0].imshow(res_oscuro, cmap="gray")
arreglo_plots[2,1].set_title('Histograma modificado')
for i,col in enumerate(colors):
    img_array_i = res_oscuro[:,:,i].ravel()
    arreglo_plots[2,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[i])

## Analicemos.

## ¿Qué sucede si variamos los valores de gamma en ambos casos?

# Ecualización del histograma.

In [ ]:
#Definir la función para aplicar la transformación sobre la imagen RGB
def apply_f_on_rgb(img, f, args):
    
    #Crear una matriz de ceros del tamaño de la imagen de entrada
    res = np.zeros(img.shape, np.uint8)
    #Aplicar la transformación f sobre cada canal del espacio de color RGB
    res[:,:,0] = f(img[:,:,0], *args)
    res[:,:,1] = f(img[:,:,1], *args)
    res[:,:,2] = f(img[:,:,2], *args)
    
    return res

#Definir la función ecualización del histograma cuyo parámetro es una imagen
def histogram_equalization(img):
     
    #Crear matriz de ceros del tamaño de la imagen y tipo de dato flotante
    res = np.zeros(img.shape, np.float32)
    #Crear un vector 1-D de la matriz de la imagen, es decir "aplanarla"
    img_raveled = img.ravel()
    #Generar el histograma normalizado de la imagen 
    hist_norm = plt.hist(img_raveled, bins=255, range=(0.0, 255.0), density=True)
    #Limpiar la figura actual
    plt.clf()
    #hist_norm[0] es un vector de probabilidades. Añadir al vector, el valor [1 - sumatoria de sus datos]
    pdf = hist_norm[0]
    np.append(pdf, 1.0 - np.sum(pdf))
    #Dado que se añade un dato, se realiza la sumatoria de valores sobre todo el vector
    cdf = [np.sum(pdf[0:x]) for x in range(0,256)]
    #Hallar el valor mínimo y máximo de la imagen
    gmin = np.min(img)
    gmax = np.max(img)
    
    #Generar F(g) - Función de ecualización
    for g in range(0,256):
        res[img == g] = (gmax - gmin)*cdf[g] + gmin
    #Asegurar que los datos sean uint8 y esten en el rango correspondiente
    res[res<0] = 0
    res[res>255] = 255
    res = res.astype(np.uint8)
    return res

#Leer la imagen y aplicar la transformación ecualización del histograma sobre ésta    
img = cv2.cvtColor(cv2.imread('res/gato.jpg'), cv2.COLOR_BGR2RGB)
res = apply_f_on_rgb(img, histogram_equalization, [])

#Generar subplot 2x3 para mostrar imagen, histograma y función de densidad acumulada
fig, arreglo_plots = plt.subplots(2,3, figsize=(20, 20))
colors = ('r','g','b')

arreglo_plots[0,0].set_title('Imagen original')
arreglo_plots[0,0].imshow(img, cmap="gray")
arreglo_plots[0,1].set_title('Histograma original')
for i,col in enumerate(colors):
    img_array_i = img[:,:,i].ravel()
    arreglo_plots[0,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[i])
arreglo_plots[0,2].set_title('Función de densidad acumulada orig.')  
for i,col in enumerate(colors):
    img_array_i = img[:,:,i].ravel()
    arreglo_plots[0,2].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[i], cumulative = True)


arreglo_plots[1,0].set_title('Imagen modificada')
arreglo_plots[1,0].imshow(res, cmap="gray")
arreglo_plots[1,1].set_title('Histograma modificado')
for i,col in enumerate(colors):
    img_array_i = res[:,:,i].ravel()
    arreglo_plots[1,1].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[i], cumulative = False)
arreglo_plots[1,2].set_title('Función de densidad acumulada modificada.')  
for i,col in enumerate(colors):
    img_array_i = res[:,:,i].ravel()
    arreglo_plots[1,2].hist(img_array_i,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[i], cumulative = True)

## Analicemos
## ¿Sería útil ecualizar el histograma de la imagen "res/gato2.jpg"?

# Convolución.

In [ ]:
#Función para hacer el kernel, su parámetro es el tamaño de éste
def make_kernel(size):
    #Matriz de unos del tamaño de entrada, datos de tipo flotante
    kernel = np.ones((size,size),dtype=np.float64)
    #r corresponde al entero más grande que sea menor o igual a size/2
    r = math.floor(size/2)
    return (r,kernel/(size**2))

#Función de aplicación de kernel sobre la imagen bajo los parámetros 
#imagen de entrada, máscara, valores en x,y,kernel,r
def apply_kernel(img, res, x,y,kernel,r):
    #Recorrer la imagen en el intervalo del radio del kernel
    for i in range(-r, r+1):
        for j in range(-r, r+1):
            #Convolución sobre cada pixel de la imagen
            res[x][y] += img[x+i][y+j] * kernel[r+i][r+j] 

#Leer la imagen
img = img_read('res/gato2.jpg', 'color')
#Tamaño del kernel
k_size = 9
#Obtener un kernel que se ajusta al tamaño solicitado mediante la función
(r, kernel) = make_kernel(k_size)
#Crear la matriz de ceros del tamaño de la imagen de entrada, tipo de datos flotante
res = np.zeros(img.shape, dtype=np.float64)

#Realizar la operación convolución sobre toda la imagen de entrada
for i in range(r,img.shape[0]-r):
    for j in range(r, img.shape[1]-r):
        apply_kernel(img, res, i, j, kernel, r)

#Asegurar el tipo de dato uint8
res=res.astype(np.uint8)

#Generar un subplot 1x2 que muestre imagen original y la convolución
fig, arreglo_plots = plt.subplots(1,2, figsize=(15, 15))

arreglo_plots[0].set_title('Imagen original')
arreglo_plots[0].imshow(img, cmap="gray", vmin=0, vmax=255)

arreglo_plots[1].set_title('Imagen convolucionada')
arreglo_plots[1].imshow(res, cmap="gray", vmin=0, vmax=255)

Utilicemos la función filter2D.

In [ ]:
#Leer la imagen de entrada
img = img_read('res/gato2.jpg', 'color')

#Crear un kernel de tamaño 5
k_size = 5
kernel = np.ones((k_size,k_size),np.float64)/k_size**2

#Aplicar filter2D sobre la imagen y el kernel arbitrario, bajo la alternativa de borde preferida
res = cv2.filter2D(img, ddepth=-1, kernel= kernel, borderType=cv2.BORDER_REPLICATE)

#Mostrar la imagen original y la imagen bajo filter2D
fig = plt.figure(figsize=(15,15))

plt.subplot(1,2,1)
plt.imshow(img,cmap='gray')
plt.title("Imagen Original")

plt.subplot(1,2,2)
plt.imshow(res,cmap='gray')
plt.title("Imagen Salida")

## Analicemos.

## ¿Por qué obtuvimos esta diferencia en los bordes de la imagen entre operaciones de convolución?

## Cambiemos el tamaño del kernel, ¿Cómo varía la imagen convolucionada?

# Tipos de borde.

Podemos seleccionar el tipo de borde que deseemos en la imagen, haciendo uso de la función de opencv.

In [ ]:
#Leer imagen de entrada
img = img_read('res/gato2.jpg', 'color')

#Asignar un valor al tamaño del borde en las 4 direcciones
#up,down,left,right
b_sz = 5

#Aplicar MakeBorder bajo todas las alternativas (último parámetro)
replicate = cv2.copyMakeBorder(img,b_sz,b_sz,b_sz,b_sz,cv2.BORDER_REPLICATE)
reflect = cv2.copyMakeBorder(img,b_sz,b_sz,b_sz,b_sz,cv2.BORDER_REFLECT)
reflect101 = cv2.copyMakeBorder(img,b_sz,b_sz,b_sz,b_sz,cv2.BORDER_REFLECT_101)
wrap = cv2.copyMakeBorder(img,b_sz,b_sz,b_sz,b_sz,cv2.BORDER_WRAP)
constant= cv2.copyMakeBorder(img,b_sz,b_sz,b_sz,b_sz,cv2.BORDER_CONSTANT,value=[0,0,0])

#Generar 6 figuras para mostrar cada una de las alternativas y la imagen original
fig = plt.figure(figsize=(15,15))
plt.subplot(231),plt.imshow(img,'gray'),plt.title('ORIGINAL')
plt.subplot(232),plt.imshow(replicate,'gray'),plt.title('REPLICATE')
plt.subplot(233),plt.imshow(reflect,'gray'),plt.title('REFLECT')
plt.subplot(234),plt.imshow(reflect101,'gray'),plt.title('REFLECT_101')
plt.subplot(235),plt.imshow(wrap,'gray'),plt.title('WRAP')
plt.subplot(236),plt.imshow(constant,'gray'),plt.title('CONSTANT')

# Tipos de ruido.

In [ ]:
#Importar librerias
%matplotlib inline

import cv2
import matplotlib.pyplot as plt
import numpy as np

#Definir la función ruido gaussiano
def gaussian_noise(img, std):
    #Las variables row, col y ch dependen de la imagen de entrada y sus componentes
    mean = 0
    row = img.shape[0]
    col = img.shape[1]
    ch =  img.shape[2] if len(img.shape) == 3 else 1       
    #Definir la matriz de ceros noisy  del mismo tamaño que la imagen original
    noisy = np.zeros(img.shape, np.float64)
    #Generar una distribución normal a partir del comando de numpy
    gauss = np.random.normal(mean,std,(row,col,ch)) if ch > 1 else np.random.normal(mean,std,(row,col))
    gauss = gauss.reshape(row,col,ch)               if ch > 1 else gauss.reshape(row,col)
    #La imagen con ruido gaussiano es la suma de la imagen de entrada con la procesada
    noisy =  img + gauss
    #Ajustar los datos para que estén entre 0 y 255
    noisy[noisy<0] = 0
    noisy[noisy>255] = 255
    return noisy

#Definir la función de ruido impulsivo
def sp_noise(img, s_vs_p, amount):
    
    #Obtener los valores de row, col y ch con base en la imagen de entrada
    row = img.shape[0]
    col = img.shape[1]
    ch =  img.shape[2] if len(img.shape) == 3 else 1  
    
    #Crear una copia de la imagen para volverla ruidosa
    noisy = img.copy()
    
    # Salt mode
    
    #Multiplica los parámetros con el tamaño de la imagen, los redondea al mínimo entero
    num_salt = np.ceil(amount * img.size * s_vs_p)
    #Genera el vector coordenadas en la imagen de copia usando la función randint de numpy
    coords = [np.random.randint(0, i - 1, int(num_salt))
          for i in img.shape]

    #La imagen evaluada en el vector coords se satura
    noisy[coords] = 255
    
    # Pepper mode
    
    #Se multiplican los parámetros por el tamaño de la imagen, variando s_vs_p
    num_pepper = np.ceil(amount* img.size * (1. - s_vs_p))
    #Generar el vector coords utilizando randint
    coords = [np.random.randint(0, i - 1, int(num_pepper))
          for i in img.shape]
    #La imagen en el vector coords se satura en color negro
    noisy[coords] = 0
    return noisy

#Definir la función ruido de Poisson
def poisson_noise(img):
    
    #Se aplica el comando de numpy random.poisson a la imagen de entrada
    noisy = np.random.poisson(img) 
    #Los valores deben estar entre 0 y 255
    noisy[noisy<0] = 0
    noisy[noisy>255] = 255
    return noisy



img = cv2.cvtColor(cv2.imread('res/nieve.jpg'), cv2.COLOR_BGR2RGB)

#Aplicar ruido de Poisson a la imagen y volver al tipo de dato uint8
pos = poisson_noise(img)
pos = pos.astype(np.uint8)

#Dar valores a los parámetros de entrada de la función ruido pimienta
s_vs_p = 0.5
amount = 0.05

#Aplicar ruido gaussiano y darle tipo de dato uint8
std = 50
res = gaussian_noise(img, std)
res = res.astype(np.uint8)
pim = sp_noise(img, s_vs_p, amount)
pim = pim.astype(np.uint8)

#Generar un subplot 1x2 para mostrar la imagen original y la imagen con ruido gaussiano
fig, arreglo_plots = plt.subplots(2,2, figsize=(20, 15))

arreglo_plots[0][0].set_title('Imagen original')
arreglo_plots[0][0].imshow(img, cmap="gray")

arreglo_plots[1][0].set_title('Imagen con ruido gaussiano')
arreglo_plots[1][0].imshow(res, cmap="gray")

arreglo_plots[0][1].set_title('Imagen con ruido pimienta')
arreglo_plots[0][1].imshow(pim, cmap="gray")

arreglo_plots[1][1].set_title('Imagen con ruido poisson')
arreglo_plots[1][1].imshow(pos, cmap="gray")

# Filtro de media

In [ ]:
#Leer la imagen de entrada
img = img_read('res/gato.jpg', 'color')

#Dar valor al tamaño del kernel
k_size = 3

#Aplicar suavizado mediante la función blur, con  un tipo de borde que puede cambiarse
res1 = cv2.blur(img,(k_size,k_size),borderType=cv2.BORDER_REPLICATE )

#Aplicar el suavizado mediante filter2D
kernel = np.ones((k_size,k_size),np.float64)/(k_size**2)
res2 = cv2.filter2D(img, ddepth= -1, kernel= kernel, borderType=cv2.BORDER_REPLICATE)

#Generar un subplot de 1x3 para mostrar imagen original, Blur y Filter2D
fig = plt.figure(figsize=(20,24))

plt.subplot(1,3,1)
plt.imshow(img,cmap='gray')
plt.title("Imagen Original")

plt.subplot(1,3,2)
plt.imshow(res1,cmap='gray')
plt.title("Función Blur")

plt.subplot(1,3,3)
plt.imshow(res2,cmap='gray')
plt.title("Función Filter2D")

## Analicemos.
## ¿Qué pasa si el kernel es más grande?

In [ ]:
#Función imagen integral
def integralImage(img):
    """
    Returns the integral image/summed area table. See here: https://en.wikipedia.org/wiki/Summed_area_table
    :param img:
    :return:
    """
    #La función cumsum de numpy permite obtener la suma acumulativa de datos 
    int_image = np.cumsum(img,0)
    int_image = np.cumsum(int_image,1)
    
    return int_image

#Función ajustar bordes
def adjustEdges(height, width, point):
    """
    This handles the edge cases if the box's bounds are outside the image range based on current pixel.
    :param height: Height of the image.
    :param width: Width of the image.
    :param point: The current point.
    :return:
    """
    #Definir un nuevo punto y verificar que esté dentro de la matriz de pixeles de la imagen
    #Así se asegura que la caja que contiene los datos esté contenida en la matriz
    newPoint = [point[0], point[1]]
    if point[0] >= height:
        newPoint[0] = height -1

    if point[1] >= width:
        newPoint[1] = width -1
    return tuple(newPoint)

#Encontrar el área de un paralelogramo
def findArea(int_img, a, b, c, d):
    """
    Finds the area for a particular square using the integral image. See summed area wiki.
    :param int_img: The
    :param a: Top left corner.
    :param b: Top right corner.
    :param c: Bottom left corner.
    :param d: Bottom right corner.
    :return: The integral image.
    """
    #El largo y el ancho del cuadrado se definen por los datos de la imagen de entrada
    height = int_img.shape[0]
    width = int_img.shape[1]
    
    #Aplicar la función ajustar bordes para asegurar que la caja esté dentro de la imagen
    a = adjustEdges(height, width, a)
    b = adjustEdges(height, width, b)
    c = adjustEdges(height, width, c)
    d = adjustEdges(height, width, d)
    
    #Condicionales que calculan la imagen integral sobre a,b,c,d si están dentro de la imagen
    a = 0 if (a[0] < 0 or a[0] >= height) or (a[1] < 0 or a[1] >= width) else int_img.item(a[0], a[1])
    b = 0 if (b[0] < 0 or b[0] >= height) or (b[1] < 0 or b[1] >= width) else int_img.item(b[0], b[1])
    c = 0 if (c[0] < 0 or c[0] >= height) or (c[1] < 0 or c[1] >= width) else int_img.item(c[0], c[1])
    d = 0 if (d[0] < 0 or d[0] >= height) or (d[1] < 0 or d[1] >= width) else int_img.item(d[0], d[1])

    return a + d - b - c

#Definir la caja que contiene los datos del filtrado
def boxFilter(img, filterSize):
    """
    Runs the subsequent box filtering steps. Prints original image, finds integral image, and then outputs final image
    :param img: An image in matrix form.
    :param filterSize: The filter size of the matrix
    :return: A final image written as finalimage.png
    """
    
    #Largo y ancho dependen de la imagen de entrada
    height = img.shape[0]
    width = img.shape[1]
    
    #Generar la imagen integral de la imagen de entrada
    intImg = integralImage(img)
    #Crear una copia de la imagen original 
    res = np.ones((height, width), np.uint64)
    
    #Encontrar el mayor número entero de la matriz filterSize/2 (radio del kernel)
    loc = int(np.floor(filterSize/2))
    #Recorrer la imagen en todas las posiciones x,y
    for y in range(height):
        for x in range(width):
            #Itemset añade un valor a la matriz en la posición y,x
            #Aplicar findArea para encontrar el área del cuadrado bajo los parámetros de entrada
            res.itemset((y, x), findArea(intImg, (y-loc-1, x-loc-1),
                                         (y-loc-1, x+loc), 
                                         (y+loc, x-loc-1), 
                                         (y+loc, x+loc))/(filterSize**2))
    return res

#Leer la imagen de entrada
img = img_read('res/gato2.jpg', 'gray')
#Tamaño del kernel
k_size = 25

#Nota, en ambos casos se asumen 0s en los bordes.

#Aplicar blur(filtro de media) y el método boxFilter en la imagen de entrada
res1 = cv2.blur(img,(k_size,k_size),borderType=cv2.BORDER_CONSTANT )
res2 = boxFilter(img, k_size)

#Generar un subplot 1x3 para mostrar imagen original y filtradas
fig, arreglo_plots = plt.subplots(1,3, figsize=(15, 15))
arreglo_plots[0].set_title('Imagen original')
arreglo_plots[0].imshow(img, cmap="gray")

arreglo_plots[1].set_title('Imagen tras filtro de media')
arreglo_plots[1].imshow(res1, cmap="gray")

arreglo_plots[2].set_title('Imagen tras box filter')
arreglo_plots[2].imshow(res2, cmap="gray")

# Filtro de mediana, su aplicación.

In [ ]:
#Leer la imagen de entrada en escala de grises
img = img_read('res/nieve.jpg', 'color')
#Aplicar ruido sal y pimienta
img = sp_noise(img, s_vs_p = 0.5, amount = 0.5)

#Definir tamaño del kernel
k_size = 5
#Aplicar filtro de mediana a la imagen de entrada
res= cv2.medianBlur(img, k_size)

#Generar un subplot 1x2 para mostrar imagen original e imagen tras filtro de mediana
fig, arreglo_plots = plt.subplots(1,2, figsize=(15, 15))
arreglo_plots[0].set_title('Imagen original')
arreglo_plots[0].imshow(img, cmap="gray")

arreglo_plots[1].set_title('Imagen tras filtro de mediana')
arreglo_plots[1].imshow(res, cmap="gray")

# Filtro de máximo y mínimo.

In [ ]:
#Definir función de Filtro del Máximo
def maximum_filter(img,k_size):
    #Crear una matriz de ceros del tamaño de la imagen de entrada
    res = np.zeros(img.shape, np.uint8)
    #Radio del kernel
    r = math.floor(k_size/2)
    #Recorrer cada posición i,j de la imagen con el kernel
    for i in range(r, img.shape[0]-r):
        for j in range(r, img.shape[1]-r):
            #Tamaño de la ventana 
            window = img[i-r:i+r+1,j-r:j+r+1]
            #Cambiar el valor de la posición i,j de la imagen por el de la ventana
            max_val = np.max(window)
            res[i,j] = max_val
    return res

#Leer imagen de entrada
img = img_read('res/nieve.jpg', 'color')

#Aplicar ruido sal y pimienta a la imagen
img = sp_noise(img, s_vs_p = 0, amount = 0.04)
#Definir tamaño del kernel
k_size = 3 

#Aplicar filtro del Máximo a la imagen
res = maximum_filter(img, k_size)
#Recordar el tipo de dato uint8, entre 0 y 255
res = res.astype(np.uint8)

#Generar un subplot 1x2 que muestre imagen original e imagen tras filtro de máximo
fig, arreglo_plots = plt.subplots(1,2, figsize=(15, 15))

arreglo_plots[0].set_title('Imagen original')
arreglo_plots[0].imshow(img, cmap="gray", vmin=0, vmax=255)

arreglo_plots[1].set_title('Imagen tras filtro de máximo')
arreglo_plots[1].imshow(res, cmap="gray", vmin=0, vmax=255)

In [ ]:
#Definir función filtro del mínimo
def minimum_filter(img,k_size):
    #Crear matriz de ceros del tamaño de imagen de entrada
    res = np.zeros(img.shape, np.uint8)
    #Definir el radio del kernel
    r = math.floor(k_size/2)
    #Recorrer cada posición i,j de la imagen con el kernel
    for i in range(r, img.shape[0]-r):
        for j in range(r, img.shape[1]-r):
            #Tamaño de la ventana 
            window = img[i-r:i+r+1,j-r:j+r+1]
            #Cambiar el valor de la posición i,j de la imagen por el de la ventana
            min_val = np.min(window)
            res[i,j] = min_val
    return res

#Leer la imagen de entrada y aplicarle ruido sal y pimienta
img = img_read('res/nieve.jpg', 'color')
img = sp_noise(img, s_vs_p = 1, amount = 0.04)
#Definir el radio del kernel
k_size = 3

#Aplicar filtro del mínimo a la imagen de entrada
res = minimum_filter(img, k_size)

#Tipo de dato entero, entre 0 y 255
res = res.astype(np.uint8)

#Generar subplot 1x2 para mostrar imagen original e imagen tras filtro del mínimo
fig, arreglo_plots = plt.subplots(1,2, figsize=(15, 15))

arreglo_plots[0].set_title('Imagen original')
arreglo_plots[0].imshow(img, cmap="gray", vmin=0, vmax=255)

arreglo_plots[1].set_title('Imagen tras filtro de mínimo')
arreglo_plots[1].imshow(res, cmap="gray", vmin=0, vmax=255)

## Analicemos
## Modifique el parámetro de la función de entrada y verifiquemos los resultados leyendo las imágenes en escala de grises.

## ¿Cómo se diferencia el ruido creado por estas funciones entre imágenes a color e imágenes en escala de grises?